In [1]:
import pandas as pd
from pathlib import Path # type: ignore
import plotly.io as pio
import sys
if (module_path:=str(Path(".").absolute().resolve().parent)) not in sys.path:
    sys.path.insert(0, module_path)
from sample_info import rename_mixtures, platforms, backgrounds, mixtures, get_mosdepth_threshold_file, plotting_dir, mixtures2drop, getHeatmap
outdir = plotting_dir / "coverage_analysis/breadth_plots"
outdir.mkdir(exist_ok=True)

In [2]:
ratios = "1,10,30,100,500,1000,5000,10000".split(",")
x_levels = [f"{x}X" for x in ratios]
def generate_threshold_dfs():
    for platform in platforms:
        for background in backgrounds:
            for mixture in mixtures:
                if mixture in mixtures2drop:
                    continue
                df = pd.read_csv(get_mosdepth_threshold_file(platform, background, mixture), sep="\t", compression='gzip')
                df["mixture"] = mixture
                df["batch"] = f"{platform}: {background}"
                df = rename_mixtures(df)
                yield df

def get_threshold_df():
    df = pd.concat((x for x in generate_threshold_dfs()))
    for x_level in x_levels:
        df[f"{x_level}_ratio"] = df[x_level]/df["0X"]
    return df

In [3]:
df = get_threshold_df()
df

,#chrom,start,end,region,0X,1X,10X,30X,100X,500X,...,mixture,batch,1X_ratio,10X_ratio,30X_ratio,100X_ratio,500X_ratio,1000X_ratio,5000X_ratio,10000X_ratio
0,MN908947.3,1,265,5'UTR,264,211,211,211,152,0,...,0adgio1o2o3o4o5,illumina_ss: wb,0.799242,0.799242,0.799242,0.575758,0.000000,0.000000,0.000000,0.000000
1,MN908947.3,1,29903,whole genome,29902,18310,17443,17320,16166,12580,...,0adgio1o2o3o4o5,illumina_ss: wb,0.612334,0.583339,0.579225,0.540633,0.420708,0.378670,0.206976,0.099291
2,MN908947.3,266,13468,ORF1a,13202,8490,8165,8042,7326,5843,...,0adgio1o2o3o4o5,illumina_ss: wb,0.643084,0.618467,0.609150,0.554916,0.442584,0.397894,0.172777,0.067869
3,MN908947.3,13468,21555,ORF1b,8087,5177,4996,4996,4952,4263,...,0adgio1o2o3o4o5,illumina_ss: wb,0.640163,0.617782,0.617782,0.612341,0.527142,0.495487,0.345616,0.214418
4,MN908947.3,21563,25384,S,3821,1217,1217,1217,1188,749,...,0adgio1o2o3o4o5,illumina_ss: wb,0.318503,0.318503,0.318503,0.310913,0.196022,0.195499,0.075896,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,MN908947.3,27394,27759,ORF7a,365,365,365,365,365,365,...,o3-4,ont: pwrb,1.000000,1.000000,1.000000,1.000000,1.000000,0.619178,0.613699,0.435616
10,MN908947.3,27894,28259,ORF8,365,365,365,365,365,365,...,o3-4,ont: pwrb,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.720548
11,MN908947.3,28274,29533,N,1259,1259,1259,1259,1259,1259,...,o3-4,ont: pwrb,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.997617
12,MN908947.3,29558,29674,ORF10,116,116,116,116,116,116,...,o3-4,ont: pwrb,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000


In [5]:
x_level = "100X"
ratio_col = f"{x_level}_ratio"
genes_of_interest = ["S", "whole genome"]
for gene in genes_of_interest:
    gene_df = df[df["region"] == gene]
    gene_df.to_csv(outdir/f"{gene.replace(' ','-')}-{x_level}-breadth-of-coverage.csv", index=False)
    fig = getHeatmap(
        gene_df,field=ratio_col,
        labels={ratio_col:f"{x_level} ratio"},title=f"Fraction of {(gene + ' gene' if len(gene) == 1 else gene).title()} Covered at {x_level}",
        title_y=0.84,
        height=800,
    )
    fig.show()
    pio.write_image(fig, outdir/f"{gene.replace(' ','-')}-{x_level}-breadth-of-coverage.jpg", width=1600, height=500, scale=2)